In [1]:
%run 01_hash.ipynb

In [2]:
import json
from copy import deepcopy

Print objects in a nice way.

In [11]:
def stringfy(o):
    s = []
    for k,v in o.__dict__.items():
        p = f'{k}:'.ljust(15)
        if hasattr(v,'messageHash'): s.append(p+ph(v.messageHash.hex()))
        elif k == 'time'           : s.append(p+time.ctime(v))
        elif str(v)[:2]=='0x'      : s.append(p+ph(v))
        elif type(v)   ==float     : s.append(p+str(round(v,8))+' eth')
        else                       : s.append(p+str(v))
    return '\n'.join(s)

## Transaction

A transaction (abbr. tx) is used for transfering `value` from one address `fr` to another `to`.

The `fee` goes to the miner who included the tx in a block. More on mining in [03_block.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/04_mine.ipynb). The higher it is, the more likely it is included in a block.

The `nonce` is the number of transactions sent from a given address. It's used to avoid replay attacks. For a more detailed explanation, see [here](https://kb.myetherwallet.com/en/transactions/what-is-nonce/). 

It's `hash` is a unique fingerprint. Every time something in the tx changes, it gets recalculated.

In [17]:
class TX(Hashable): 
    def __init__(self, fr, to, value, fee, nonce): 
        self.fr, self.to = fr, to
        self.value       = float(value)
        self.fee         = fee
        self.nonce       = nonce
        self.time        = time.time()
        self.signed      = False
        
    def __setattr__(self, prop, val):
        super().__setattr__(prop, val)
        if prop not in ['sig','signed']: super().__setattr__('hash', sha(self.__dict__))
        if prop == 'sig'               : self.signed = True
    
    def json(self):
        d = {**self.__dict__}
        d['time'] = time.ctime(d['time'])
        return json.dumps(d, indent=4)
    
    def smry(self): return f'{pmh(self.fr)} -> {pmh(self.to)} {self.value} eth'
    def __str__(self): return stringfy(self)

Create random txs.

In [18]:
tx1 = TX(rh(), rh(), 12, 0.2,  0)
tx2 = TX(rh(), rh(), 6,  0.15, 0); tx1.smry()

'🕐 0xeaa5 -> 🕔 0xee96 12.0 eth'

In [20]:
print(tx1.json())

{
    "fr": "0xeaa547f3f2e6f3b65dec93fd657e7d034e7a64f75b1979173d0af42440df7afd",
    "hash": "0xd1416ffe13e66350d10d50e9174ad580615ff033128a9623c8974e6798e03690",
    "to": "0xee9678c5a074d265028972aa87655db62825972b0cf8a9ab3c13106466279773",
    "value": 12.0,
    "fee": 0.2,
    "nonce": 0,
    "time": "Fri Apr  2 22:15:32 2021",
    "signed": false
}


In [22]:
def txs2str(txs): return '\n'.join([str(tx)+'\n' for tx in txs])
print(txs2str([tx1,tx2]))

fr:            🕐 0xeaa547f3f2...afd
hash:          🔷 0xd1416ffe13...690
to:            🕔 0xee9678c5a0...773
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Fri Apr  2 22:15:32 2021
signed:        False

fr:            💵 0x4fc8057ab1...3ae
hash:          📒 0x6c0bf30e4e...b2d
to:            📖 0x70d0b2892d...a5a
value:         6.0 eth
fee:           0.15 eth
nonce:         0
time:          Fri Apr  2 22:15:32 2021
signed:        False



#### Changing Tx
Every change in the object is reflected by its hash. Compare the tx hash below with the hash of unchanged tx above. The tx hash has changed. This is how we can make sure that nobody changes the tx without us knowing.

In [320]:
tx2_false_value       = deepcopy(tx2)
tx2_false_value.value = 120
print(tx2_false_value)

fr:            🔬 0xc6dfed7bbc...669
hash:          📔 0x6e325ed6a2...ee6
to:            📾 0x9884fcdd1d...dea
value:         120
fee:           0.15 eth
nonce:         0
time:          Wed Mar 31 10:11:00 2021
signed:        False


Transactions can be determined unequal by simply comparing the hashes as implemented in `__eq__`.

In [321]:
assert tx2 != tx2_false_value

#### Signing Tx
An account (implemented in [02_account.ipynb](https://github.com/SharifElfouly/blockchain-from-scratch/blob/main/02_account.ipynb)) has the ability to sign a tx. Here we simply mock a signature with a random hash. The signature `sig` is saved as an attribute.

In [322]:
tx1_signed = deepcopy(tx1)
tx1.sig    = rh(); print(tx1)

fr:            📜 0x7654aeb8c4...1c0
hash:          🔴 0xce23d9df63...a10
to:            💛 0x358a84a7ad...c30
value:         12.0 eth
fee:           0.2 eth
nonce:         0
time:          Wed Mar 31 10:11:00 2021
signed:        True
sig:           🔾 0xd85e32e03e...7e0


Signing the tx should not change its hash. Therefore it is equal to the unsigned tx.

In [323]:
assert tx1 == tx1_signed